In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, 
                                                  fixed_length_right=40, 
                                                  remove_stop_words=True)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7519.77it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4090.68it/s]
Building Vocabulary from a datapack.: 100%|██████████| 234263/234263 [00:00<00:00, 2887381.76it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7689.51it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4079.95it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 106145.05it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 122/122 [00:00<00:00, 7577.45it/s]
Processing text_right with chain_transfor

In [4]:
model = mz.models.MatchPyramid()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['optimizer'] = 'adam'
model.params['dropout_rate'] = 0.1

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.match_pyramid.MatchPyramid'>
input_shapes                  [(10,), (40,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           16546
embedding_output_dim          300
embedding_trainable           True
num_blocks                    2
kernel_count                  [16, 32]
kernel_size                   [[3, 3], [3, 3]]
activation                    relu
dpool_size                    [3, 10]
padding                       same
dropout_rate                  0.1


In [5]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             4963800     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_layer_1 (MatchingLayer (None, 10, 40, 1)    0           embedding[0][0]                  
          

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [7]:
model.load_embedding_matrix(embedding_matrix)

In [8]:
dpool_callback = mz.data_generator.callbacks.DynamicPooling(
    fixed_length_left=10, 
    fixed_length_right=40
)
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    callbacks=[dpool_callback]
)
print('num batches:', len(train_generator))

num batches: 102


In [9]:
test_generator = mz.DataGenerator(
    test_pack_processed,
    batch_size=20,
    callbacks=[dpool_callback]
)
len(test_generator)

118

In [10]:
test_x, test_y = test_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_y))

In [11]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/20
102/102 [==============================] - 7s 64ms/step - loss: 0.7424
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5139454549140169 - normalized_discounted_cumulative_gain@5(0.0): 0.5834297539119333 - mean_average_precision(0.0): 0.5352191588484061
Epoch 2/20
102/102 [==============================] - 12s 113ms/step - loss: 0.4619
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5210624357736955 - normalized_discounted_cumulative_gain@5(0.0): 0.5835136157485148 - mean_average_precision(0.0): 0.5441885150393843
Epoch 3/20
102/102 [==============================] - 11s 110ms/step - loss: 0.2771
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5236275738408916 - normalized_discounted_cumulative_gain@5(0.0): 0.5894045008193476 - mean_average_precision(0.0): 0.5434810442721834
Epoch 4/20
102/102 [==============================] - 11s 110ms/step - loss: 0.1936
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5170996303251092 - norma

In [23]:
model.evaluate(test_x, test_y, batch_size=21)

{normalized_discounted_cumulative_gain@3(0.0): 0.5446226903757947,
 normalized_discounted_cumulative_gain@5(0.0): 0.6095009668628633,
 mean_average_precision(0.0): 0.5623808717746093}

In [24]:
append_params_to_readme(model)